In [400]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import prophet
import itertools
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.serialize import model_to_json, model_from_json

In [401]:
dataset_path = '../data/'

pollen_test = pd.read_csv(dataset_path + 'new_pollen_test.csv')
pollen_val = pd.read_csv(dataset_path + 'new_pollen_val.csv')
pollen_train = pd.read_csv(dataset_path + 'new_pollen_train.csv')
pollen_type = pd.read_csv(dataset_path + 'pollen-type.csv')
submission_example = pd.read_csv(dataset_path + 'submission_example.csv')
weather_data = pd.read_csv(dataset_path + 'weather_data.csv')

In [402]:
print(max(pollen_train.AMBROSIA))

237.5878048780488


In [403]:
# make index column
pollen_train['date_idx'] = [i for i in range(len(pollen_train))]
pollen_val['date_idx'] = [i for i in range(len(pollen_val))]
pollen_test['date_idx'] = [i for i in range(len(pollen_test))]

In [404]:
pollen_train.drop(['date'], axis=1, inplace=True)
pollen_val.drop(['date'], axis=1, inplace=True)
pollen_test.drop(['date'], axis=1, inplace=True)

In [405]:
pollen_train.shape

(279, 27)

In [406]:
pollen_test.shape

(112, 28)

## Model

In [407]:
import warnings
import copy
warnings.filterwarnings('ignore')

In [408]:
columns = pollen_train.columns
batch_id = pollen_test['batch_id']

In [409]:
batch_id

0        1
1        2
2        3
3        4
4        5
      ... 
107    108
108    109
109    110
110    111
111    112
Name: batch_id, Length: 112, dtype: int64

In [410]:
ans_columns = pd.DataFrame(columns=['batch_id', '1 day prediction', '2 days prediction', '3 days prediction'])
ans_columns['batch_id'] = [val for val in batch_id]

In [411]:
pollen_test_copy = copy.deepcopy(pollen_test)

for idx, column in enumerate(columns):
    if column == 'date_idx':
        continue
    batch_id = pollen_test.drop(['batch_id'], axis=1)
    test_column = pollen_test.drop([column], axis=1)
    
    model = xgb.XGBRegressor()

    train_data = copy.deepcopy(pollen_train).drop([column], axis=1)
    model.fit(train_data, pollen_val[column])

    col_ans = model.predict(pollen_test.iloc[:, ~pollen_test.columns.isin([column, 'batch_id'])])

    if column == 'AMBROSIA':
        ans_columns['1 day prediction'] = [int(val) for val in col_ans]

    pollen_test_copy[column] = (pollen_test_copy[column] + col_ans) / 2

pollen_test = copy.deepcopy(pollen_test_copy)

In [412]:
pollen_test_copy = copy.deepcopy(pollen_test)

for idx, column in enumerate(columns):
    if column == 'date_idx':
        continue
    batch_id = pollen_test.drop(['batch_id'], axis=1)
    test_column = pollen_test.drop([column], axis=1)
    
    model = xgb.XGBRegressor()

    train_data = copy.deepcopy(pollen_train).drop([column], axis=1)
    model.fit(train_data, pollen_val[column])

    col_ans = model.predict(pollen_test.iloc[:, ~pollen_test.columns.isin([column, 'batch_id'])])

    if column == 'AMBROSIA':
        ans_columns['2 days prediction'] = [int(val) for val in col_ans]

    pollen_test_copy[column] = (pollen_test_copy[column] + col_ans) / 2

pollen_test = copy.deepcopy(pollen_test_copy)

In [413]:
pollen_test_copy = copy.deepcopy(pollen_test)

for idx, column in enumerate(columns):
    if column == 'date_idx':
        continue
    batch_id = pollen_test.drop(['batch_id'], axis=1)
    test_column = pollen_test.drop([column], axis=1)
    
    model = xgb.XGBRegressor()

    train_data = copy.deepcopy(pollen_train).drop([column], axis=1)
    model.fit(train_data, pollen_val[column])

    col_ans = model.predict(pollen_test.iloc[:, ~pollen_test.columns.isin([column, 'batch_id'])])

    if column == 'AMBROSIA':
        ans_columns['3 days prediction'] = [int(val) for val in col_ans]

    pollen_test_copy[column] = (pollen_test_copy[column] + col_ans) / 2

pollen_test = copy.deepcopy(pollen_test_copy)

In [414]:
ans_columns[70:90]

,batch_id,1 day prediction,2 days prediction,3 days prediction
70,71,14,33,40
71,72,13,11,32
72,73,2,2,2
73,74,92,75,185
74,75,58,28,34
75,76,170,74,177
76,77,169,62,149
77,78,3,11,11
78,79,29,19,53
79,80,3,2,3


In [415]:
# Save to csv

ans_columns.to_csv('../results/xgboost.csv', index=False)

In [416]:
# read csv file 

prophet = pd.read_csv('../results/prophet_by_town.csv')

In [417]:
ans_columns = (ans_columns + prophet) / 2

In [418]:
# convert to int ans_columns

ans_columns = ans_columns.astype(int)

In [421]:
ans_columns[70:90]

,batch_id,1 day prediction,2 days prediction,3 days prediction
70,71,39,53,58
71,72,27,30,46
72,73,31,35,40
73,74,113,119,187
74,75,29,14,17
75,76,118,77,135
76,77,93,41,86
77,78,59,65,65
78,79,81,77,96
79,80,47,47,49


In [420]:
ans_columns.to_csv('../results/ensemble.csv', index=False)